In [ ]:
import pandas as pd
import numpy as np
df = pd.read_csv('online_retail.csv')  
df.head()

Invoice StockCode                          Description  Quantity  \
0  489434     85048  15CM CHRISTMAS GLASS BALL 20 LIGHTS        12   
1  489434    79323P                   PINK CHERRY LIGHTS        12   
2  489434    79323W                  WHITE CHERRY LIGHTS        12   
3  489434     22041         RECORD FRAME 7" SINGLE SIZE         48   
4  489434     21232       STRAWBERRY CERAMIC TRINKET BOX        24   

           InvoiceDate  Price  Customer ID         Country  
0  12/01/2009 07:45:00   6.95      13085.0  United Kingdom  
1  12/01/2009 07:45:00   6.75      13085.0  United Kingdom  
2  12/01/2009 07:45:00   6.75      13085.0  United Kingdom  
3  12/01/2009 07:45:00   2.10      13085.0  United Kingdom  
4  12/01/2009 07:45:00   1.25      13085.0  United Kingdom

In [25]:
df.info()
df.describe(include='all')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 525461 entries, 0 to 525460
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   Invoice      525461 non-null  object 
 1   StockCode    525461 non-null  object 
 2   Description  522533 non-null  object 
 3   Quantity     525461 non-null  int64  
 4   InvoiceDate  525461 non-null  object 
 5   Price        525461 non-null  float64
 6   Customer ID  417534 non-null  float64
 7   Country      525461 non-null  object 
dtypes: float64(2), int64(1), object(5)
memory usage: 32.1+ MB


Invoice StockCode                         Description       Quantity  \
count   525461    525461                              522533  525461.000000   
unique   28816      4632                                4681            NaN   
top     537434    85123A  WHITE HANGING HEART T-LIGHT HOLDER            NaN   
freq       675      3516                                3549            NaN   
mean       NaN       NaN                                 NaN      10.337667   
std        NaN       NaN                                 NaN     107.424110   
min        NaN       NaN                                 NaN   -9600.000000   
25%        NaN       NaN                                 NaN       1.000000   
50%        NaN       NaN                                 NaN       3.000000   
75%        NaN       NaN                                 NaN      10.000000   
max        NaN       NaN                                 NaN   19152.000000   

                InvoiceDate          Price    Customer ID         Country  
count                525461  525461.000000  417534.000000          525461  
unique                25296            NaN            NaN              40  
top     12/06/2010 16:57:00            NaN            NaN  United Kingdom  
freq                    675            NaN            NaN          485852  
mean                    NaN       4.688834   15360.645478             NaN  
std                     NaN     146.126914    1680.811316             NaN  
min                     NaN  -53594.360000   12346.000000             NaN  
25%                     NaN       1.250000   13983.000000             NaN  
50%                     NaN       2.100000   15311.000000             NaN  
75%                     NaN       4.210000   16799.000000             NaN  
max                     NaN   25111.090000   18287.000000             NaN

In [26]:
df.rename(columns={'Customer ID': 'CustomerID'}, inplace=True)

In [27]:
df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate'])
df['CustomerID'] = df['CustomerID'].astype('Int64')   # allows NaN

df.drop_duplicates(inplace=True)

In [28]:
df.head()

Invoice StockCode                          Description  Quantity  \
0  489434     85048  15CM CHRISTMAS GLASS BALL 20 LIGHTS        12   
1  489434    79323P                   PINK CHERRY LIGHTS        12   
2  489434    79323W                  WHITE CHERRY LIGHTS        12   
3  489434     22041         RECORD FRAME 7" SINGLE SIZE         48   
4  489434     21232       STRAWBERRY CERAMIC TRINKET BOX        24   

          InvoiceDate  Price  CustomerID         Country  
0 2009-12-01 07:45:00   6.95       13085  United Kingdom  
1 2009-12-01 07:45:00   6.75       13085  United Kingdom  
2 2009-12-01 07:45:00   6.75       13085  United Kingdom  
3 2009-12-01 07:45:00   2.10       13085  United Kingdom  
4 2009-12-01 07:45:00   1.25       13085  United Kingdom

In [29]:
df['Sales'] = df['Quantity'] * df['Price']

In [30]:
df['Month'] = df['InvoiceDate'].dt.to_period('M')

monthly_sales = df.groupby('Month').agg(
    Total_Sales=('Sales', 'sum'),
    Total_Orders=('Invoice', 'nunique'),
    Total_Items=('Quantity', 'sum')
).reset_index()

monthly_sales['Month'] = monthly_sales['Month'].astype(str)
monthly_sales.head()

Month  Total_Sales  Total_Orders  Total_Items
0  2009-12   796648.500          2330       418570
1  2010-01   622479.502          1633       374510
2  2010-02   531265.366          1969       367501
3  2010-03   763247.241          2367       488012
4  2010-04   587926.692          1892       350478

In [31]:
filtered_and = df[(df['Quantity'] > 10) & (df['Price'] > 20)]

In [32]:
df['Description_clean'] = (
    df['Description']
    .astype(str)
    .str.lower()
    .str.strip()
    .str.replace('[^a-zA-Z0-9 ]', '', regex=True)
)

In [33]:
df['Year'] = df['InvoiceDate'].dt.year
df['MonthNum'] = df['InvoiceDate'].dt.month
df['Day'] = df['InvoiceDate'].dt.day
df['Weekday'] = df['InvoiceDate'].dt.day_name()

In [35]:
pivot_sales = pd.pivot_table(
    df,
    values='Sales',
    index='Country',
    columns='Year',
    aggfunc='sum',
    fill_value=0
)
pivot_sales.head()

Year          2009      2010
Country                     
Australia    50.25  30001.55
Austria    1998.34  11024.94
Bahrain       0.00   2313.15
Belgium     447.60  22196.43
Bermuda       0.00   1253.14

In [36]:
customer_df = df.dropna(subset=['CustomerID']).copy()

In [38]:
snapshot_date = df['InvoiceDate'].max() + pd.Timedelta(days=1)

rfm = customer_df.groupby('CustomerID').agg(
    Recency=('InvoiceDate', lambda x: (snapshot_date - x.max()).days),
    Frequency=('Invoice', 'nunique'),
    Monetary=('Sales', 'sum')
).reset_index()
rfm.head()

CustomerID  Recency  Frequency  Monetary
0       12346       67         15    -51.74
1       12347        3          2   1323.32
2       12348       74          1    222.16
3       12349       43          4   2646.99
4       12351       11          1    300.93

In [39]:
top_customers = rfm.sort_values('Monetary', ascending=False).head(10)
top_customers

CustomerID  Recency  Frequency   Monetary
4254       18102        1         95  341776.73
1662       14646       10         87  243853.05
1291       14156        7        138  183163.55
1869       14911        1        270  137710.76
957        13694        9        105  128172.42
3807       17511        3         42   83760.60
1983       15061        3         90   82162.74
3178       16684       15         34   75610.17
511        13089        4        132   55801.73
3228       16754        8         35   54558.18